In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

texts_train = np.random.randint(0, 100, (1000, 50))
labels_train = np.random.randint(0, 5, 1000)
texts_test = np.random.randint(0, 100, (200, 50))
labels_test = np.random.randint(0, 5, 200)

train_data = TensorDataset(torch.tensor(texts_train, dtype=torch.long), torch.tensor(labels_train, dtype=torch.long))
test_data = TensorDataset(torch.tensor(texts_test, dtype=torch.long), torch.tensor(labels_test, dtype=torch.long))

batch_size = 400
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

class TicketClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, target_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(embed_dim, target_size)

    def forward(self, text):
        embedded = self.embedding(text).permute(0, 2, 1)
        conved = torch.relu(self.conv(embedded)).mean(dim=2)
        return self.fc(conved)

vocab_size = 101
target_size = 5
embedding_dim = 64

model = TicketClassifier(vocab_size, embedding_dim, target_size)

lr = 0.05
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

epochs = 3
model.train()
for i in range(epochs):
    running_loss, num_processed = 0, 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        num_processed += len(inputs)
    print(f"Epoch: {i+1}, Loss: {running_loss/num_processed}")

def calculate_metrics(predictions, labels, num_classes):
    accuracy = (predictions == labels).sum().item() / len(labels)
    precision, recall = [], []
    for i in range(num_classes):
        tp = ((predictions == i) & (labels == i)).sum().item()
        fp = ((predictions == i) & (labels != i)).sum().item()
        fn = ((predictions != i) & (labels == i)).sum().item()
        precision.append(tp / (tp + fp) if tp + fp != 0 else 0.0)
        recall.append(tp / (tp + fn) if tp + fn != 0 else 0.0)
    return accuracy, precision, recall

model.eval()
all_predictions, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        output = model(inputs)
        cat = torch.argmax(output, dim=-1)
        all_predictions.extend(cat.tolist())
        all_labels.extend(labels.tolist())

all_predictions = torch.tensor(all_predictions)
all_labels = torch.tensor(all_labels)

accuracy, precision, recall = calculate_metrics(all_predictions, all_labels, target_size)
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)
